In [ ]:
# Importing libraries
import json
import time
from telnetlib import Telnet
import serial
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Initializing keys, PORT and telnet configuration

ser = serial.Serial('COM3', 9600, timeout=1);
tn=Telnet('localhost',13854);

start=time.clock();

i=0;
tn.write(b'{"enableRawOutput": true, "format": "Json"}');

eSenseDict={'attention':0, 'meditation':0};
waveDict={'lowGamma':0, 'highGamma':0, 'highAlpha':0, 'delta':0, 'highBeta':0, 'lowAlpha':0, 'lowBeta':0, 'theta':0};
signalLevel=0;
signalValue = "";

open_data = []
close_data = []
fingopen= 1;
fingclose = 1;

def hzconvert(s):
    return s[-5:2]

In [1]:
EEGDATAFRAME = pd.DataFrame(columns=['Time', 'Attention', 'Meditation','Blink_Strength'])
WAVE_EEGDATAFRAME = pd.DataFrame(columns=['lowGamma', 'highGamma','highAlpha','delta','highBeta','lowAlpha','lowBeta','theta'])

while i<100:
    eeg_data = []
    eeg_data_wave = []

    blinkStrength=0;

    line=tn.read_until(b'\r');
    
    if len(line) > 20:
        timediff=time.clock()-start;
        dict=json.loads((line));
        
        if "poorSignalLevel" in dict:
            signalLevel=dict['poorSignalLevel'];
        if "blinkStrength" in dict:
            blinkStrength=dict['blinkStrength'];
        if "eegPower" in dict:
            waveDict=dict['eegPower'];
            eSenseDict=dict['eSense'];

        attentionStr=str(eSenseDict['attention']);
        meditationStr=str(eSenseDict['meditation']);
        
        lowGamma=hzconvert(str(waveDict['lowGamma']));
        highGamma=hzconvert(str(waveDict['highGamma']));
        highAlpha=hzconvert(str(waveDict['highAlpha']));
        delta=hzconvert(str(waveDict['delta']));
        highBeta=hzconvert(str(waveDict['highBeta']));
        lowAlpha=hzconvert(str(waveDict['lowAlpha']));
        lowBeta=hzconvert(str(waveDict['lowBeta']));
        theta=hzconvert(str(waveDict['theta']));
        
        blinkStrengthStr=str(blinkStrength);
        seconds = time.time();
        
    
        if(signalLevel==200):
            signalValue = "Poor";
        else:
            signalValue = "Strong";
            
        
        if(signalValue == "Strong"):
            attentionInt = int(attentionStr)
            meditationInt = int(meditationStr)

            if ((attentionInt > 69) or (meditationInt > 50)):
                close_data.append(attentionInt)
                if(len(close_data) > 3 ):
                    movements = "Close"
                    eeg_data_wave.append({'lowGamma': lowGamma,'highGamma': highGamma,'highAlpha': highAlpha, 'delta':delta, 'highBeta':highBeta, 'lowAlpha':lowAlpha, 'lowBeta':lowBeta, 'theta':theta, 'attention':attention, 'meditation':meditation, 'movements':movements})    
                    eeg_wave_df = pd.DataFrame(eeg_data_wave)
                    WAVE_EEGDATAFRAME = WAVE_EEGDATAFRAME.append(eeg_wave_df, ignore_index=True)
                    WAVE_EEGDATAFRAME.to_csv('wave_eegdata.csv',index=True)
                    
                    ser.write(str.encode('b'));
                    print("Close Fingers");
                    time.sleep(1)

            elif ((attentionInt < 70) and (attentionInt > 10)):
                open_data.append(attentionInt)
                if(len(open_data) > 3 ):
                    movements = "Open"
                    eeg_data_wave.append({'lowGamma': lowGamma,'highGamma': highGamma,'highAlpha': highAlpha, 'delta':delta, 'highBeta':highBeta, 'lowAlpha':lowAlpha, 'lowBeta':lowBeta, 'theta':theta,'movements':movements})    
                    eeg_wave_df = pd.DataFrame(eeg_data_wave)
                    WAVE_EEGDATAFRAME = WAVE_EEGDATAFRAME.append(eeg_wave_df, ignore_index=True)
                    WAVE_EEGDATAFRAME.to_csv('wave_eegdata.csv',index=True)
                    
                    ser.write(str.encode('a'));
                    print("Open Fingers");
                    time.sleep(1)
                
            else:
                open_data.clear()
                close_data.clear()
                print("Out of control");

        else:
            print("Poor Signal");
        
        

            
ser.close();
tn.close();

NameError: name 'pd' is not defined